# Análisis Exploratorio de Datos - Landmarks de Actividades Humanas

Este notebook analiza los landmarks extraídos de videos usando MediaPipe Pose.

**Objetivos:**
- Entender la estructura de los datos
- Visualizar trayectorias de landmarks
- Identificar características distintivas por actividad
- Detectar problemas en los datos (missing values, ruido, etc.)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("Librerías importadas correctamente")

## 1. Cargar Datos

Primero cargamos los landmarks extraídos. Si aún no has procesado videos, corre:
```bash
python ../src/data/extract_landmarks.py --input tu_video.mp4 --output ../data/processed/landmarks.csv
```

In [ ]:
# Ruta a los datos procesados
data_path = Path('../data/processed/landmarks.csv')

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"✓ Datos cargados: {len(df)} frames")
    print(f"✓ Columnas: {len(df.columns)}")
    display(df.head())
else:
    print("❌ No se encontraron datos procesados.")
    print(f"   Busca en: {data_path}")
    print("\nPrimero extrae landmarks con el script extract_landmarks.py")

## 2. Estadísticas Básicas

In [ ]:
# Información general
print("=== INFO GENERAL ===")
print(f"Total frames: {len(df)}")
print(f"Duración (30fps): {len(df)/30:.2f} segundos")
print(f"\nPrimeros/últimos timestamps:")
print(f"  Inicio: {df['timestamp'].min():.2f}s")
print(f"  Fin: {df['timestamp'].max():.2f}s")

In [ ]:
# Calidad del tracking
visibility_cols = [col for col in df.columns if col.endswith('_visibility')]

print("\n=== CALIDAD DE TRACKING ===")
print(f"\nVisibility promedio por landmark:")

visibility_summary = df[visibility_cols].mean().sort_values(ascending=False)
print(visibility_summary.head(10))

print(f"\n✓ Landmarks con visibility > 0.9: {(visibility_summary > 0.9).sum()}")
print(f"⚠ Landmarks con visibility < 0.7: {(visibility_summary < 0.7).sum()}")

## 3. Visualización de Visibility por Zona del Cuerpo

In [ ]:
# Agrupar landmarks por zona
zones = {
    'Cabeza': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear'],
    'Torso': ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip'],
    'Brazos': ['left_elbow', 'right_elbow', 'left_wrist', 'right_wrist'],
    'Piernas': ['left_knee', 'right_knee', 'left_ankle', 'right_ankle']
}

zone_visibility = {}
for zone, landmarks in zones.items():
    visibility_cols_zone = [f'{lm}_visibility' for lm in landmarks]
    zone_visibility[zone] = df[visibility_cols_zone].mean().mean()

# Gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(zone_visibility.keys(), zone_visibility.values(), color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A'])
plt.ylabel('Visibility Promedio', fontsize=12)
plt.title('Calidad de Tracking por Zona del Cuerpo', fontsize=14, fontweight='bold')
plt.ylim(0, 1)
plt.axhline(y=0.9, color='green', linestyle='--', alpha=0.5, label='Excelente (>0.9)')
plt.axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, label='Aceptable (>0.7)')
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Trayectorias de Landmarks Clave

Visualizamos cómo se mueven las articulaciones principales a lo largo del tiempo.

In [ ]:
# Landmarks clave para visualizar
key_landmarks = ['nose', 'left_shoulder', 'right_shoulder', 
                 'left_hip', 'right_hip', 'left_knee', 'right_knee']

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Trayectoria en X (horizontal)
for landmark in key_landmarks:
    axes[0].plot(df['timestamp'], df[f'{landmark}_x'], label=landmark, alpha=0.7)
axes[0].set_ylabel('Coordenada X (normalizada)', fontsize=11)
axes[0].set_title('Movimiento Horizontal de Landmarks', fontsize=13, fontweight='bold')
axes[0].legend(loc='best', ncol=2)
axes[0].grid(alpha=0.3)

# Trayectoria en Y (vertical)
for landmark in key_landmarks:
    axes[1].plot(df['timestamp'], df[f'{landmark}_y'], label=landmark, alpha=0.7)
axes[1].set_xlabel('Tiempo (segundos)', fontsize=11)
axes[1].set_ylabel('Coordenada Y (normalizada)', fontsize=11)
axes[1].set_title('Movimiento Vertical de Landmarks', fontsize=13, fontweight='bold')
axes[1].legend(loc='best', ncol=2)
axes[1].grid(alpha=0.3)
axes[1].invert_yaxis()  # En video, Y=0 es arriba

plt.tight_layout()
plt.show()

## 5. Análisis de Velocidades

Calculamos la velocidad de movimiento de articulaciones clave.

In [ ]:
def calculate_velocity(df, landmark, coord='x'):
    """Calcula velocidad de un landmark en una coordenada."""
    col = f'{landmark}_{coord}'
    velocity = df[col].diff() / df['timestamp'].diff()
    return velocity.fillna(0)

# Calcular velocidades para landmarks clave
df['left_wrist_vel_x'] = calculate_velocity(df, 'left_wrist', 'x')
df['right_wrist_vel_x'] = calculate_velocity(df, 'right_wrist', 'x')
df['left_knee_vel_y'] = calculate_velocity(df, 'left_knee', 'y')
df['right_knee_vel_y'] = calculate_velocity(df, 'right_knee', 'y')
df['nose_vel_x'] = calculate_velocity(df, 'nose', 'x')

# Visualizar
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Velocidades horizontales de muñecas
axes[0].plot(df['timestamp'], df['left_wrist_vel_x'], label='Muñeca Izq', alpha=0.7)
axes[0].plot(df['timestamp'], df['right_wrist_vel_x'], label='Muñeca Der', alpha=0.7)
axes[0].set_ylabel('Velocidad X', fontsize=11)
axes[0].set_title('Velocidad Horizontal de Muñecas', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Velocidades verticales de rodillas
axes[1].plot(df['timestamp'], df['left_knee_vel_y'], label='Rodilla Izq', alpha=0.7)
axes[1].plot(df['timestamp'], df['right_knee_vel_y'], label='Rodilla Der', alpha=0.7)
axes[1].set_xlabel('Tiempo (segundos)', fontsize=11)
axes[1].set_ylabel('Velocidad Y', fontsize=11)
axes[1].set_title('Velocidad Vertical de Rodillas', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Cálculo de Ángulos de Articulaciones

Los ángulos entre articulaciones son features muy útiles para clasificación.

In [ ]:
def calculate_angle(p1, p2, p3):
    """
    Calcula el ángulo en p2 formado por p1-p2-p3.
    
    Args:
        p1, p2, p3: Arrays de coordenadas [x, y]
    Returns:
        Ángulo en grados
    """
    v1 = p1 - p2
    v2 = p3 - p2
    
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)

# Calcular ángulo de rodilla izquierda (cadera-rodilla-tobillo)
angles_left_knee = []
for idx, row in df.iterrows():
    hip = np.array([row['left_hip_x'], row['left_hip_y']])
    knee = np.array([row['left_knee_x'], row['left_knee_y']])
    ankle = np.array([row['left_ankle_x'], row['left_ankle_y']])
    
    angle = calculate_angle(hip, knee, ankle)
    angles_left_knee.append(angle)

df['left_knee_angle'] = angles_left_knee

# Visualizar
plt.figure(figsize=(14, 5))
plt.plot(df['timestamp'], df['left_knee_angle'], linewidth=2, color='#E74C3C')
plt.xlabel('Tiempo (segundos)', fontsize=11)
plt.ylabel('Ángulo (grados)', fontsize=11)
plt.title('Ángulo de Rodilla Izquierda a lo Largo del Tiempo', fontsize=13, fontweight='bold')
plt.axhline(y=180, color='green', linestyle='--', alpha=0.5, label='Pierna recta (180°)')
plt.axhline(y=90, color='orange', linestyle='--', alpha=0.5, label='Ángulo recto (90°)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nÁngulo promedio de rodilla: {df['left_knee_angle'].mean():.1f}°")
print(f"Ángulo mínimo: {df['left_knee_angle'].min():.1f}°")
print(f"Ángulo máximo: {df['left_knee_angle'].max():.1f}°")

## 7. Detección de Frames Problemáticos

In [ ]:
# Frames con baja visibility
threshold = 0.5
key_landmarks_vis = [f'{lm}_visibility' for lm in key_landmarks]

df['min_visibility'] = df[key_landmarks_vis].min(axis=1)
problematic_frames = df[df['min_visibility'] < threshold]

print(f"=== FRAMES PROBLEMÁTICOS ===")
print(f"Total frames: {len(df)}")
print(f"Frames con visibility < {threshold}: {len(problematic_frames)} ({len(problematic_frames)/len(df)*100:.1f}%)")

if len(problematic_frames) > 0:
    print(f"\nFrames problemáticos se encuentran en:")
    print(problematic_frames[['frame', 'timestamp', 'min_visibility']].head(10))

# Visualizar visibility mínima a lo largo del tiempo
plt.figure(figsize=(14, 5))
plt.plot(df['timestamp'], df['min_visibility'], linewidth=2, color='#3498DB')
plt.axhline(y=threshold, color='red', linestyle='--', label=f'Umbral ({threshold})')
plt.xlabel('Tiempo (segundos)', fontsize=11)
plt.ylabel('Visibility Mínima', fontsize=11)
plt.title('Calidad de Tracking a lo Largo del Video', fontsize=13, fontweight='bold')
plt.ylim(0, 1)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Resumen y Conclusiones

In [ ]:
print("=" * 60)
print("RESUMEN DEL ANÁLISIS EXPLORATORIO")
print("=" * 60)

print(f"\n1. DATOS GENERALES")
print(f"   - Total frames procesados: {len(df)}")
print(f"   - Duración del video: {len(df)/30:.2f} segundos")
print(f"   - Landmarks por frame: 33")

print(f"\n2. CALIDAD DE TRACKING")
for zone, vis in zone_visibility.items():
    status = "✓" if vis > 0.9 else "⚠" if vis > 0.7 else "❌"
    print(f"   {status} {zone}: {vis:.3f}")

print(f"\n3. FRAMES PROBLEMÁTICOS")
print(f"   - Frames con baja visibility: {len(problematic_frames)} ({len(problematic_frames)/len(df)*100:.1f}%)")

print(f"\n4. FEATURES CALCULADAS")
print(f"   ✓ Velocidades (muñecas, rodillas, nariz)")
print(f"   ✓ Ángulo de rodilla")
print(f"   ✓ Visibility por zona")

print(f"\n5. PRÓXIMOS PASOS")
print(f"   → Normalizar coordenadas")
print(f"   → Filtrar ruido en tracking")
print(f"   → Interpolar frames con baja visibility")
print(f"   → Extraer más features (inclinación torso, distancias, etc.)")
print(f"   → Segmentar actividades")

print("\n" + "=" * 60)

## Notas Finales

Este notebook proporciona un análisis básico de los landmarks extraídos. Para la Entrega 2, necesitaremos:

1. **Ampliar el análisis** a múltiples videos de diferentes actividades
2. **Comparar actividades** para identificar características distintivas
3. **Implementar preprocesamiento** (normalización, filtrado, feature engineering)
4. **Preparar datos para ML** (crear matrices de features, labels, splits)

**Para ejecutar este notebook:**
```bash
cd entrega1/notebooks
jupyter notebook exploratory_analysis.ipynb
```